In [5]:
import pandas as pd
import numpy as np

# Load the data
movies_df = pd.read_csv('moviedata/movies.csv', header=0)
ratings_df = pd.read_csv('moviedata/ratings.csv', header=0)

In [6]:
smaller_df = ratings_df.sample(n=100000, random_state=1)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.optimizers import Adam

# Assuming your DataFrame has the columns 'userId', 'movieId', and 'rating'
num_users = smaller_df['userId'].nunique()
num_movies = smaller_df['movieId'].nunique()

# Model architecture
user_input = Input(shape=(1,), name='user_input', dtype='int64')
user_embedding = Embedding(num_users + 1, 15, name='user_embedding')(user_input)
user_vec = Flatten(name='user_flatten')(user_embedding)

movie_input = Input(shape=(1,), name='movie_input', dtype='int64')
movie_embedding = Embedding(num_movies + 1, 15, name='movie_embedding')(movie_input)
movie_vec = Flatten(name='movie_flatten')(movie_embedding)

concat = Concatenate()([user_vec, movie_vec])

fc1 = Dense(128, activation='relu')(concat)
fc2 = Dense(32, activation='relu')(fc1)
output = Dense(1)(fc2)

model = Model([user_input, movie_input], output)

# Compile the model
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Split your data into a training and test set
from sklearn.model_selection import train_test_split

X = smaller_df[['userId', 'movieId']].values
y = smaller_df['rating'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit([X_train[:, 0], X_train[:, 1]], y_train, 
                    validation_data=([X_test[:, 0], X_test[:, 1]], y_test), 
                    batch_size=64, 
                    epochs=5)

# Evaluate the model
model.evaluate([X_test[:, 0], X_test[:, 1]], y_test)


AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)